In [1]:
import numpy as np
import pandas as pd
import random
import copy
import networkx as nx
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import adjusted_mutual_info_score

In [2]:
def game_community_opinion(W, S_RANDOM, X_INI, random_choose, Lambda = 1.5, BETA = 1, gamma = 0.95, ETA=0.1):
    """
    ################################ 输入参数 ###############################
        Influence matrix W : n*n, 
        initial opinion vector X_INI : n*1, 
        equilibrium initial community membership matrix S_INI : n*c,
        diagonal matrix Lambda_MATRIX : n*n,
        tuning parameters BETA and gamma.
    """
    t = 0
    S_STAR = S_RANDOM
    row_identity = np.ones((n, 1))
    ALL_ONES = np.ones((n, n))
    
    Lambda_MATRIX = Lambda*np.eye(n)
    
    ############### 节点i的邻居节点 ： W_ij + W_ji > 0 and i #####################
    Neighbor_set = []
    Neighbor_MATRIX = W + W.T + np.eye(n)
    for i in range(n):
        Neighbor_set.append(list(np.where(Neighbor_MATRIX[i])[0]))
        
    FINAL_PHI_S = 0
    
    while 1:
        #print('########################### t = %d ######################'%t)
        
        E = X_INI*row_identity.T - row_identity*X_INI.T
        E = np.abs(E)
        D = ETA - E
        B = np.maximum(0, D)
        
        C = np.matmul(Lambda_MATRIX, D) - B
        Y = C*W+C.T*W.T
        
        Z = Y.reshape(n, 1, n)
        
        S_INI = copy.deepcopy(S_STAR)
        S_Y = np.matmul(S_INI.T, Y)
        S_Y_S = np.matmul(S_Y, S_INI)
        PHI_S = 0.5*S_Y_S.trace()
        
        DELTA_PHI = 1e10
        
        ################################  更新社区标签  ################################
        while DELTA_PHI!=0:
            DELTA_PHI = 0
            for i in range(n):
                F_i = S_INI[Neighbor_set[i]]
                S_S = np.matmul(F_i, S_INI.T)
                P_i = np.matmul(S_S, Z[i].T)
                
                # V_i = P_i.argmax()
                if np.random.rand() < random_choose:
                    # 随机选择一个邻居索引
                    V_i = np.random.randint(0, len(P_i))
                else:
                    # 正常贪心选择
                    V_i = int(np.argmax(P_i))
                
                S_i_star = F_i[V_i]

                S_MINUS_S_S = np.matmul(S_i_star - S_INI[i], S_INI.T)
                DELTA_PHI += np.matmul(S_MINUS_S_S, Z[i].T)
                S_INI[i] = S_i_star
                            
            PHI_S += DELTA_PHI[0]
            #print("PHI_S : %d"%PHI_S)
            
        beta_gamma = BETA*pow(gamma, t)
        Xi_t = beta_gamma*ALL_ONES
        relu_D = np.maximum(0, Xi_t - E)
        W_D = W*relu_D
        
        X_S_S = np.matmul(S_INI, S_INI.T)
        Lambda_MODIFY = Lambda*ALL_ONES - 1
        X_S_S_MODIFY = 1 + Lambda_MODIFY*X_S_S
        
        THETA = X_S_S_MODIFY*W_D
        
        GAMMA = np.matmul(THETA, ALL_ONES) + ALL_ONES
        PHI = THETA / GAMMA
        
        PHI_prime = np.matmul(THETA, row_identity) + row_identity
        PHI_prime = row_identity / PHI_prime
        
        X_UPDATE = PHI_prime*X_INI + np.matmul(PHI, X_INI)
        
        S_STAR = S_INI
        
#         print("PHI_S : ",PHI_S)
#         print("FINAL_PHI_S : ",FINAL_PHI_S)
        
        DETA_X = np.abs(X_UPDATE - X_INI)
        if DETA_X.max() < 1e-3:
            break
        
        X_INI = X_UPDATE
        FINAL_PHI_S = PHI_S
        
        t += 1
    
    return S_STAR, X_UPDATE, PHI

In [3]:
"""
##############################  全局变量  #############################
n : 节点数量
c : 初始社区数量
"""
row = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4,5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7, 8, 8, 8, 8, 8, 9, 9, 10, 10,10, 11, 12, 12, 13, 13, 13, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17,18, 18, 19, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 23, 23, 23, 24,24, 24, 25, 25, 25, 26, 26, 27, 27, 27, 27, 28, 28, 28, 29, 29, 29,29, 30, 30, 30, 30, 31, 31, 31, 31, 31, 31, 32, 32, 32, 32, 32, 32,32, 32, 32, 32, 32, 32, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33,33, 33, 33, 33, 33, 33]
col = [1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 17, 19, 21, 31, 0, 2, 3, 7,13, 17, 19, 21, 30, 0, 1, 3, 7, 8, 9, 13, 27, 28, 32, 0, 1, 2, 7,12, 13, 0, 6, 10, 0, 6, 10, 16, 0, 4, 5, 16, 0, 1, 2, 3, 0, 2, 30,32, 33, 2, 33, 0, 4, 5, 0, 0, 3, 0, 1, 2, 3, 33, 32, 33, 32, 33, 5,6, 0, 1, 32, 33, 0, 1, 33, 32, 33, 0, 1, 32, 33, 25, 27, 29, 32,33, 25, 27, 31, 23, 24, 31, 29, 33, 2, 23, 24, 33, 2, 31, 33, 23,26, 32, 33, 1, 8, 32, 33, 0, 24, 25, 28, 32, 33, 2, 8, 14, 15, 18,20, 22, 23, 29, 30, 31, 33, 8, 9, 13, 14, 15, 18, 19, 20, 22, 23,26, 27, 28, 29, 30, 31, 32]
W = np.zeros((34, 34))
for i in range(len(row)):
    W[row[i]][col[i]]=1

n = W.shape[0]
ALL_ONES = np.ones((n, n))

G_COMPUTE_Q = nx.from_numpy_array(W)

COMMUNITY_LABELS = np.array([1,2,3,4,5,6,7,8,11,12,13,14,17,18,20,22]) - 1
TRUE_COMMUNITY = np.zeros(n)
TRUE_COMMUNITY[list(COMMUNITY_LABELS)] = 1

c = 20

In [4]:
################################# 给定初始社区结构
S = np.zeros((n, c))
random.seed(62)
community_index = [random.randint(0, c-1) for _ in range(n)]
for i in range(n):
    S[i][community_index[i]]=1

random_o_all_ami = []
random_o_all_ari = []

for random_value in [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09]: 
    o_all_ami = []
    o_all_ari = []
    for x_seed in range(100):
        np.random.seed(x_seed)
        X = np.random.uniform(0, 1, [n, 1])
        
        FINAL_S, FINAL_X, FINAL_WEIGHT = game_community_opinion(W, S, X, random_value, Lambda = 1.2, ETA = 0.4, gamma = 0.9)
    
        NODES_LABELS = np.where(FINAL_S==1)[1]
        COMMUNITY_LABELS = list(set(NODES_LABELS))
        
        COMMUNITY_PROPAGATION = []
        for i in range(len(COMMUNITY_LABELS)):
            COMMUNITY_LABEL_SET = []
            COMMUNITY_LABEL_ONE = COMMUNITY_LABELS[i]
            for j in range(n):
                if NODES_LABELS[j]==COMMUNITY_LABEL_ONE:
                    COMMUNITY_LABEL_SET.append(j)
            COMMUNITY_PROPAGATION.append(COMMUNITY_LABEL_SET)
        
        ari = adjusted_rand_score(TRUE_COMMUNITY, NODES_LABELS)
        ami = adjusted_mutual_info_score(TRUE_COMMUNITY, NODES_LABELS)
        o_all_ari.append(ari)
        o_all_ami.append(ami)
        
    print("ari mean: ", np.array(o_all_ari).mean())
    print("ami mean: ", np.array(o_all_ami).mean())
    random_o_all_ami.append(o_all_ami)
    random_o_all_ari.append(o_all_ari)

ari mean:  0.6868549590055096
ami mean:  0.6841006756431979
ari mean:  0.635484081689296
ami mean:  0.6373602669150847
ari mean:  0.6084124529328068
ami mean:  0.6080151725475905
ari mean:  0.5686658929539241
ami mean:  0.5755110240083646
ari mean:  0.4271434120609198
ami mean:  0.4334569134782489
ari mean:  0.43868307468219286
ami mean:  0.44122758324372635
ari mean:  0.3687006382716095
ami mean:  0.38072256996820025
ari mean:  0.32632002829962486
ami mean:  0.33229918485546167
ari mean:  0.2805883375065423
ami mean:  0.28570375116403973
ari mean:  0.23188583433079923
ami mean:  0.2438799035670339


In [5]:
######################## 给定初始向量
np.random.seed(4)
X = np.random.uniform(0, 1, [n, 1])

random_c_all_ami = []
random_c_all_ari = []

for random_value in [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09]:
    c_all_ami = []
    c_all_ari = []
    for c_seed in range(100):
        S = np.zeros((n, c))
        random.seed(c_seed)
        community_index = [random.randint(0, c-1) for _ in range(n)]
        for i in range(n):
            S[i][community_index[i]]=1
        
        FINAL_S, FINAL_X, FINAL_WEIGHT = game_community_opinion(W, S, X, random_value, Lambda = 1.2, ETA = 0.2, gamma = 0.9)

        NODES_LABELS = np.where(FINAL_S==1)[1]
        COMMUNITY_LABELS = list(set(NODES_LABELS))
        
        COMMUNITY_PROPAGATION = []
        for i in range(len(COMMUNITY_LABELS)):
            COMMUNITY_LABEL_SET = []
            COMMUNITY_LABEL_ONE = COMMUNITY_LABELS[i]
            for j in range(n):
                if NODES_LABELS[j]==COMMUNITY_LABEL_ONE:
                    COMMUNITY_LABEL_SET.append(j)
            COMMUNITY_PROPAGATION.append(COMMUNITY_LABEL_SET)
        
        ari = adjusted_rand_score(TRUE_COMMUNITY, NODES_LABELS)
        ami = adjusted_mutual_info_score(TRUE_COMMUNITY, NODES_LABELS)
        c_all_ari.append(ari)
        c_all_ami.append(ami)
        
    print("ari mean: ", np.array(c_all_ari).mean())
    print("ami mean: ", np.array(c_all_ami).mean())
    random_c_all_ami.append(c_all_ami)
    random_c_all_ari.append(c_all_ari)

ari mean:  0.9960549279837363
ami mean:  0.9963689742748847
ari mean:  0.977588609793045
ami mean:  0.9794818197726431
ari mean:  0.9253189569031107
ami mean:  0.9333609959667746
ari mean:  0.9458538682032704
ami mean:  0.9505208105851726
ari mean:  0.8797360367970032
ami mean:  0.8893106195047941
ari mean:  0.7987998855604687
ami mean:  0.8082179098059207
ari mean:  0.7385810320930156
ami mean:  0.748398878967555
ari mean:  0.6554059716841769
ami mean:  0.6681719089422355
ari mean:  0.6689883436281187
ami mean:  0.6784117691356509
ari mean:  0.6089012293415174
ami mean:  0.6234092588571071


In [6]:
karate_random_o_all_ari = np.array(random_o_all_ari)
karate_random_c_all_ari = np.array(random_c_all_ari)

karate_random_o_all_ari = pd.DataFrame(karate_random_o_all_ari)

# 保存为 Excel 文件
karate_random_o_all_ari.to_excel('karate_random_o_all_ari.xlsx', index=False, header=False)

karate_random_c_all_ari = pd.DataFrame(karate_random_c_all_ari)

# 保存为 Excel 文件
karate_random_c_all_ari.to_excel('karate_random_c_all_ari.xlsx', index=False, header=False)